<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [4]:
## Task 1: Mark all launch sites on a map
# Create a DataFrame with launch site coordinates
launch_sites_df = pd.DataFrame({
    'Site': ['Launch Site 1', 'Launch Site 2', 'Launch Site 3', 'Launch Site 4'],
    'Latitude': [28.573255, 34.632093, 28.562302, 29.559684],
    'Longitude': [-80.646895, -120.610829, -80.577356, -95.083097]
})

# Create a folium map centered at the first launch site
site_map = folium.Map(location=[launch_sites_df['Latitude'].mean(), launch_sites_df['Longitude'].mean()], zoom_start=5)

# Add each launch site to the map
for index, row in launch_sites_df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Site'],
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(site_map)

# Display the map
site_map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [11]:
# Use correct column names: Lat and Long
nasa_coordinate = [launch_sites_df['Lat'].mean(), launch_sites_df['Long'].mean()]

# Define the color that will be used for both the circle and the text
marker_color = '#d35400'

# Create a folium map centered at NASA's coordinate
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add a Circle and a Marker
for index, row in launch_sites_df.iterrows():
    # Add a circle with a 1000m radius using the same color as the marker text
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,
        color=marker_color,
        fill=True,
        fill_color=marker_color
    ).add_child(folium.Popup(row['Launch Site'])).add_to(site_map)

    # Add a marker with a label that matches the circle color
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:%s;"><b>%s</b></div>' % (marker_color, row['Launch Site']),
        )
    ).add_to(site_map)

# Display the map
site_map



The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line? -
        *   The Equator is located at 0 degrees latitude. Launch sites closer to the Equator benefit from the rotational speed of the Earth, which gives rockets a boost when launching into space.
        
        We can analyze the latitude of each launch site:
        
        Latitudes of launch sites:
        Launch Site 1: 28.573255 (Close to Cape Canaveral, Florida)
        Launch Site 2: 34.632093 (Likely Vandenberg Space Force Base, California)
        Launch Site 3: 28.562302 (Close to Cape Canaveral, Florida)
        Launch Site 4: 29.559684 (Likely SpaceX launch site in Texas)
        Findings:
        None of the launch sites are directly on the Equator (0° latitude).
        However, the latitudes of some launch sites, such as those near Cape Canaveral and Texas, are relatively close to the Equator compared to those farther north, such as Vandenberg in California.
        Launch sites near the Equator provide an advantage by utilizing the Earth's rotational speed to boost rockets into orbit, reducing fuel consumption and increasing payload efficiency.



            While not all launch sites are near the Equator, those that are closer (such as in Florida and Texas) take advantage of the Earth's rotation for efficient launches.

*   Are all launch sites in very close proximity to the coast?

        *   Let’s consider the launch site locations in relation to coastlines:
        
        Proximity to coasts:
        Launch Site 1: Close to Cape Canaveral, Florida (directly on the coast)
        Launch Site 2: Vandenberg Space Force Base, California (near the Pacific Ocean)
        Launch Site 3: Close to Cape Canaveral, Florida (directly on the coast)
        Launch Site 4: SpaceX Boca Chica, Texas (directly on the Gulf Coast)
        Findings:
        All the launch sites are very close to coastal areas. This proximity to the coast is by design.
        Launching rockets over the ocean is a safety measure. In the event of a failure during launch, debris is more likely to fall into the ocean rather than populated areas, minimizing risk to human life and property.
        Additionally, launching over water allows rockets to more easily reach the necessary flight trajectory to orbit without having to navigate over land, which can add complexity to the flight path.

            All launch sites are in close proximity to the coast for safety and trajectory optimization reasons.
            

Also please try to explain your findings.


In [12]:
# Task 2: Mark the success/failed launches for each site on the map
# Group the data by 'Launch Site' and 'class' to get the number of successes and failures
launch_outcomes = spacex_df.groupby(['Launch Site', 'class']).size().unstack(fill_value=0)
launch_outcomes.reset_index(inplace=True)

# Display the result to check the counts of successes and failures
print(launch_outcomes)

# Define the colors for success (1) and failure (0)
success_color = '#28a745'  # Green for success
failure_color = '#dc3545'  # Red for failure

# Create a folium map centered at NASA's coordinate
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add a Circle for successes and failures
for index, row in launch_outcomes.iterrows():
    # Get the launch site details
    site_name = row['Launch Site']
    lat = launch_sites_df[launch_sites_df['Launch Site'] == site_name]['Lat'].values[0]
    long = launch_sites_df[launch_sites_df['Launch Site'] == site_name]['Long'].values[0]
    
    # Add a circle for successful launches
    if row[1] > 0:
        folium.Circle(
            location=[lat, long],
            radius=1000 * row[1],  # Radius proportional to number of successful launches
            color=success_color,
            fill=True,
            fill_color=success_color,
            fill_opacity=0.6,
            popup=f"{site_name}: {row[1]} Successes"
        ).add_to(site_map)

    # Add a circle for failed launches
    if row[0] > 0:
        folium.Circle(
            location=[lat, long],
            radius=1000 * row[0],  # Radius proportional to number of failed launches
            color=failure_color,
            fill=True,
            fill_color=failure_color,
            fill_opacity=0.6,
            popup=f"{site_name}: {row[0]} Failures"
        ).add_to(site_map)

    # Add a marker with the launch site name
    folium.Marker(
        location=[lat, long],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#000000;"><b>%s</b></div>' % site_name,
        )
    ).add_to(site_map)

# Display the map
site_map


class   Launch Site   0   1
0       CCAFS LC-40  19   7
1      CCAFS SLC-40   4   3
2        KSC LC-39A   3  10
3       VAFB SLC-4E   6   4


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [13]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [14]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [18]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red


# Map the class column to predefined folium icon colors
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Initialize a MarkerCluster object
marker_cluster = MarkerCluster()

# Create a folium map centered at NASA's coordinate
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add the marker cluster to the map
marker_cluster = MarkerCluster().add_to(site_map)

# For each launch record, add a marker to the MarkerCluster
for index, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.Icon(color=row['marker_color']),  # Now using 'green' and 'red'
        popup=f"Launch Site: {row['Launch Site']}\nClass: {'Success' if row['class'] == 1 else 'Failure'}"
    ).add_to(marker_cluster)

# Display the map
site_map


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [38]:
import folium
from folium.plugins import MarkerCluster

# Create a folium map centered at NASA's coordinate
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Initialize a MarkerCluster object for individual launches
marker_cluster = MarkerCluster()

# Add the marker cluster to the map
marker_cluster.add_to(site_map)

# Add the orange circle overlay for each launch site with a smaller radius
for index, row in launch_sites_df.iterrows():
    # Create a smaller orange circle with a radius of 50 meters
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=50,  # Set radius to 50 meters for better appearance when zoomed in
        color='#FFA500',  # Orange color for the circle
        fill=True,
        fill_color='#FFA500',
        fill_opacity=0.2,  # Transparent enough for better visibility
        popup=f"Launch Site: {row['Launch Site']}"  # Add the popup for launch site name
    ).add_to(site_map)

# For each launch record, add a marker to the MarkerCluster for individual launch events
for index, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.Icon(color=row['marker_color'], icon='info-sign'),
        popup=f"Launch Site: {row['Launch Site']}\nOutcome: {'Success' if row['class'] == 1 else 'Failure'}"
    ).add_to(marker_cluster)

# Display the map
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [40]:
# TASK 3: Calculate the distances between a launch site to its proximities
import math

# Function to calculate haversine distance between two coordinates
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radius of Earth in kilometers
    
    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    
    # Differences in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # Distance in kilometers
    distance = R * c
    return distance

# Coordinates of the launch site (example: CCAFS SLC-40)
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# List of target locations (example: nearby cities or landmarks)
targets = [
    {"name": "Kennedy Space Center Visitor Complex", "lat": 28.5721, "lon": -80.6480},
    {"name": "Orlando", "lat": 28.538336, "lon": -81.379234},
    {"name": "Miami", "lat": 25.761680, "lon": -80.191790}
]

# Calculate the distance from the launch site to each target
for target in targets:
    distance = haversine_distance(launch_site_lat, launch_site_lon, target["lat"], target["lon"])
    print(f"Distance from {target['name']} to the launch site: {distance:.2f} km")



Distance from Kennedy Space Center Visitor Complex to the launch site: 6.98 km
Distance from Orlando to the launch site: 78.37 km
Distance from Miami to the launch site: 313.74 km


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [41]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [42]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [ ]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [66]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [69]:
import folium
from folium.plugins import MarkerCluster

# Create a folium map centered at NASA's coordinate
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Initialize a MarkerCluster object for individual launches
marker_cluster = MarkerCluster()

# Add the marker cluster to the map
marker_cluster.add_to(site_map)

# Add the orange circle overlay for each launch site with a smaller radius
for index, row in launch_sites_df.iterrows():
    # Create a smaller orange circle with a radius of 50 meters
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=50,  # Set radius to 50 meters for better appearance when zoomed in
        color='#FFA500',  # Orange color for the circle
        fill=True,
        fill_color='#FFA500',
        fill_opacity=0.2,  # Transparent enough for better visibility
        popup=f"Launch Site: {row['Launch Site']}"  # Add the popup for launch site name
    ).add_to(site_map)

# For each launch record, add a marker to the MarkerCluster for individual launch events
for index, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.Icon(color=row['marker_color'], icon='info-sign'),
        popup=f"Launch Site: {row['Launch Site']}\nOutcome: {'Success' if row['class'] == 1 else 'Failure'}"
    ).add_to(marker_cluster)

#Work out distance to coastline
coordinates = [
    [28.56342, -80.57674],
    [28.56342, -80.56756]]

lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)
distance = calculate_distance(coordinates[0][0], coordinates[0][1], coordinates[1][0], coordinates[1][1])
distance_circle = folium.Marker(
    [28.56342, -80.56794],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
        )
    )

site_map.add_child(distance_circle)
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [72]:
import folium
from folium.plugins import MarkerCluster

# Create a folium map centered at NASA's coordinate
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Initialize a MarkerCluster object for individual launches
marker_cluster = MarkerCluster()

# Add the marker cluster to the map
marker_cluster.add_to(site_map)

# Add the orange circle overlay for each launch site with a smaller radius
for index, row in launch_sites_df.iterrows():
    # Create a smaller orange circle with a radius of 50 meters
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=50,  # Set radius to 50 meters for better appearance when zoomed in
        color='#FFA500',  # Orange color for the circle
        fill=True,
        fill_color='#FFA500',
        fill_opacity=0.2,  # Transparent enough for better visibility
        popup=f"Launch Site: {row['Launch Site']}"  # Add the popup for launch site name
    ).add_to(site_map)

# For each launch record, add a marker to the MarkerCluster for individual launch events
for index, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.Icon(color=row['marker_color'], icon='info-sign'),
        popup=f"Launch Site: {row['Launch Site']}\nOutcome: {'Success' if row['class'] == 1 else 'Failure'}"
    ).add_to(marker_cluster)
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

distance_railway = calculate_distance(28.55768, -80.80188, 28.573255, -80.646895)
distance_railway

# Display the distance between railway point and launch site using the icon property 

site_map.add_child(folium.Marker(location=[28.55768, -80.80188],
                               popup='15.239',
                               icon=folium.Icon(color='blue', icon_color='gray')))

site_map.add_child(distance_marker)
site_map



After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?


Railways: Launch sites are usually located relatively close to railways to facilitate the transportation of heavy equipment and rockets.
Highways: There is close proximity to highways to ensure access for transportation needs.
Coastlines: Almost all launch sites are located very close to coastlines for safety during rocket launches.
Cities: Launch sites are located far from cities to avoid any potential danger from launch failures.

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
